# In this Jupyter notebook we demonstrate how to build a python Predictive Model with Scikit-learn.

The Dataset for personal loan classification is taken from: https://www.kaggle.com/itsmesunil/bank-loan-modelling

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
import seaborn as sns
%matplotlib inline
sns.set(style="ticks")

from scipy.stats import zscore
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection

In [ ]:
# Insert Cloud Object Storage Credentials and Load Dataset here.

In [ ]:
# Replace the credentials that you got from Watson Machine Learning service

wml_credentials = {
  "apikey": "<api key>",
  "instance_id": "<instance id>",
  "url": "<URL>"
}

In [ ]:
data.columns = ["ID","Age","Experience","Income","ZIPCode","Family","CCAvg","Education","Mortgage","PersonalLoan","SecuritiesAccount","CDAccount","Online","CreditCard"]
data.columns

### Exploring the dataset

#### The dataset has 5000 rows of data and 14 attributes

In [ ]:
data.shape

In [ ]:
data.info()

#### No columns have null data in the file

In [ ]:
data.apply(lambda x : sum(x.isnull()))

#### Eye balling the data

In [ ]:
data.describe().transpose()

#### Finding unique data

In [ ]:
data.apply(lambda x: len(x.unique()))

#### There are 52 records with negative experience. Before proceeding any further we need to clean the same

In [ ]:
data[data['Experience'] < 0]['Experience'].count()

#### Clean the negative variable

In [ ]:
dfExp = data.loc[data['Experience'] >0]
negExp = data.Experience < 0
column_name = 'Experience' 

#### Getting the customer ID who has negative experience

In [ ]:
mylist = data.loc[negExp]['ID'].tolist()

#### There are 52 records with negative experience

In [ ]:
negExp.value_counts()

#### So we Remove the negative experience records

In [ ]:
for id in mylist:
    age = data.loc[np.where(data['ID']==id)]["Age"].tolist()[0]
    education = data.loc[np.where(data['ID']==id)]["Education"].tolist()[0]
    df_filtered = dfExp[(dfExp.Age == age) & (dfExp.Education == education)]
    exp = df_filtered['Experience'].median()
    data.loc[data.loc[np.where(data['ID']==id)].index, 'Experience'] = exp

#### Verify records with negative experience are there or not

In [ ]:
data[data['Experience'] < 0]['Experience'].count()

In [ ]:
data.describe().transpose()

In [ ]:
sns.boxplot(x='Education',y='Income',hue='PersonalLoan',data=data)

**Observation** : It seems the customers whose education level is 1 is having more income. However customers who has taken the personal loan have the same income levels

In [ ]:
sns.boxplot(x="Education", y='Mortgage', hue="PersonalLoan", data=data,color='yellow')

**Inference** : From the above chart it seems that customer who do not have personal loan and customer who has personal loan have high mortgage

In [ ]:
sns.countplot(x="SecuritiesAccount", data=data,hue="PersonalLoan")

**Observation** : Majority of customers who does not have loan have securities account

In [ ]:
sns.countplot(x='Family',data=data,hue='PersonalLoan',palette='Set1')

**Observation** : Family size does not have any impact in personal loan. But it seems families with size of 3 are more likely to take loan. When considering future campaign this might be good association.

In [ ]:
sns.countplot(x='CDAccount',data=data,hue='PersonalLoan')

**Observation** : Customers who does not have CD account , does not have loan as well. This seems to be majority. But almost all customers who has CD account has loan as well

In [ ]:
sns.boxplot(x=data.Family,y=data.Income,hue=data.PersonalLoan)

**Observation** : Looking at the above plot, families with income less than 100K are less likely to take loan, than families with high income

In [ ]:
print('Credit card spending of Non-Loan customers: ',data[data.PersonalLoan == 0]['CCAvg'].median()*1000)
print('Credit card spending of Loan customers    : ', data[data.PersonalLoan == 1]['CCAvg'].median()*1000)

### Develop a Naive Bayes Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

Split the data in to Training(70%) and Testing(30%) 

In [ ]:
train_set, test_set = train_test_split(data.drop(['ID','Experience'], axis=1), test_size=0.3 , random_state=100)

In [ ]:
train_labels = train_set.pop('PersonalLoan')
test_labels = test_set.pop('PersonalLoan')

#### Train the Model and get Predictions

In [ ]:
naive_model = GaussianNB()
naive_model.fit(train_set, train_labels)

prediction = naive_model.predict(test_set)
naive_model.score(test_set,test_labels)

#### The model scores an accuracy of 88.67%

In [ ]:
print(prediction)

# Deploy the model to Watson Machine Learning

In [ ]:
!pip install watson-machine-learning-client

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
instance_details = client.service_instance.get_details()

In [ ]:
published_model = client.repository.store_model(model=naive_model, meta_props={'name':'Personal Loan Prediction Model'}, \
                                                training_data=train_set, training_target=train_labels)

In [ ]:
import json
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

print(json.dumps(model_details, indent=2))

In [ ]:
models_details = client.repository.list_models()

In [ ]:
loaded_model = client.repository.load(published_model_uid)

In [ ]:
test_predictions = loaded_model.predict(test_set[:10])

In [ ]:
# The predictions made by the model.
print(test_predictions)

In [ ]:
created_deployment = client.deployments.create(published_model_uid, 'Deployment of Personal Loan Prediction model')

In [ ]:
deployments = client.deployments.get_details()
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)

In [ ]:
print(scoring_endpoint)

In [ ]:
#Age	Income	ZIPCode	Family	CCAvg	Education	Mortgage	SecuritiesAccount	CDAccount	Online	CreditCard
#39	139	95616	3	3.4	1	483	0	0	1	0
#29	31	92126	4	0.3	2	0	0	0	1	0

scoring_payload = { "fields":["Age","Income","ZIPCode","Family","CCAvg","Education","Mortgage","SecuritiesAccount","CDAccount","Online", "CreditCard"],"values":[[39,139,95616,3,3.4,1,483,0,0,1,0]]}
# scoring_payload = { "fields":["Age","Income","ZIPCode","Family","CCAvg","Education","Mortgage","SecuritiesAccount","CDAccount","Online", "CreditCard"],"values":[[29,31,92126,4,0.3,2,0,0,0,1,0]]}

In [ ]:
predictions = client.deployments.score(scoring_endpoint, scoring_payload)

In [ ]:
print(json.dumps(predictions, indent=2))

In [ ]:
print(predictions['values'][0][0])